# 05-3 트리의 앙상블
### 앙상블 학습의 이해
---
</br>

💡용어 정리💡  
❓ 정형 데이터 ➡️ 특정 구조로 되어 있는 데이터 (csv, database, excel 등)  
 --> 정형 데이터를 다루는 알고리즘 중 최고 성과 '앙상블 학습'  
</br>

❓ 앙상블 학습 ➡️  
- 대부분 결정 트리 기반  
- 가장 대표적 --> 랜덤 포레스트  
</br>

❓ 랜덤 포레스트 ➡️  
- 사용자가 입력한 훈련 데이터에서 랜덤하게 샘플 추출(**중복 추출 가능!!**) --> 부트스트랩 샘플  
 --> 부트스트랩 샘플 크기 = 훈련 세트 크기  
- 랜덤한 샘플 셀렉 --> 훈련 세트에 과대적합되는 것을 맞아줌  
 --> 따라서, 검증 세트와 테스트 세트에서 안정적인 성능!
 1. RandomForestClassifier --> 전체 특성 개수의 제곱근 만큼의 특성을 랜덤하게 선택  
 2. RandomForestRegressor --> 전체 특성 사용  

❓ OOB 샘플 ➡️ 부트스트랩 샘플에 포함되지 않고 남는 샘플  
 --> 위 샘플을 이용하여 훈련된 결정트리 평가 가능! => 검증 세트의 역할 수행  

❓ 엑스트라 트리 ➡️ 랜덤 포레스트와 매우 비슷함  
 --> 다른점?? 부트스트램 샘플을 사용하지 않음, 노드를 분할할 때 무작위로 분할! (랜덤 포레스트는 가장 좋은 분할을 탐색한 후 분할) 

❓ 그레이디언트 부스팅 ➡️ 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블 하는 방법  
 --> sklearn.GradientBoosting : 깊이가 3인 결정트리 100개 사용  
 - 경사 하강법을 이용하여 트리를 앙상블에 추가함  
 - 분류 -> 로지스틱 손실 함수, 회귀 -> 평균 제곱 오차 함수 사용  
 - 학습률 매개변수로 속도를 조절함  
 - 훈련 속도가 느림 --> 순서대로 트리를 추가하므로  

❓ 히스토그램 기반 그레이디언트 부스팅 ➡️ 정형 데이터를 다루는 머신러닝 알고리즘 중에 가장 인기가 높은 알고리즘!  
 1. 입력 특성을 256개의 구간으로 분할 


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

### RandomForest

**교차 검증 수행 - cross_validate()**

In [3]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)

# return_train_score=True --> 훈련 세트에 대한 점수도 같이 반환(기존은 검증 점수만 제공! 기본 False)
# 과대적합 파악에 용이!
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

# 훈련 세트에 과대적합?
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


**특성 중요도**
- 이전에 만든 특성 중요도보다 비교적 균등한 점수?  
 --> 랜덤 포레스트가 특성의 일부를 랜덤하게 선택했기 때문!  
 --> 하나의 특성에 과도하게 집중하지xx -> 더 많은 특성이 훈련에 기여할 기회 부여  
 **==> 과대적합 감소, 일반화 성능 증가**

In [4]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


**OOB 점수를 사용한 모델 훈련**
- 교차 검증 대신 가능!  
- 훈련 세트에 더 많은 샘플 사용 가능  

In [5]:
# oob_score=True --> 각 결정 트리의 oob 점수를 평균하여 출력
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


### ExtraTree
- 랜덤 포레스트보다 빠른 속도!  
 --> 결정 트리는 최적의 분할을 찾는 데 시간을 많이 소모함, 따라서 단순 무작위로 구성한다면 훨씬 빨라짐  

In [6]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


**특성 중요도 측정**
- 엑스트라 트리 또한 랜덤 포레스트와 마찬가지로 결정 트리보다 당도에 대한 의존성이 낮음!

In [7]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


### GradientBoosting  
- 과대적합에 강함 --> 깊이가 얕은 결정 트리를 사용하기 때문  
- 일반적으로 높은 일반화 성능 기대!  

In [8]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [9]:
# 결정 트리 개수 500개로 증대
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
# 교차 검증 점수 확인
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [10]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


### Histogram-based Gradient Boosting

In [11]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 과대적합을 잘 억제하면서, 그레이디언트 부스팅 보다 조금 더 높은 성능!

/usr/local/lib/python3.7/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:17: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  "Since version 1.0, "


0.9321723946453317 0.8801241948619236


**훈련 세트를 이용한 특성 중요도 측정**

In [12]:
# 히스토그램 기반 그레이디언트 부스팅의 특성 중요도 계산하기 위함 함수 임포트 --> permutation_importance()
# 특성을 하나씩 랜덤하게 섞음 --> 모델의 성능 변화 관찰 ==> 어떤 특정이 중요한지 체크!
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
# n_repeats --> 램덤하게 섞을 횟수 지정
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

# (특성 중요도, 평균, 표준 편차) 순으로 출력

[0.08876275 0.23438522 0.08027708]


**테스트 세트를 이용한 특성 중요도 측정**

In [13]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [14]:
hgb.score(test_input, test_target) # 히스토그램 기반 그레이디언트 부스팅을 사용한 테스트 세트의 점수

0.8723076923076923

#### XGBoost
**사이킷런 이외에도 히스토그램 기반 그레이디언트 부스팅 알고리즘을 구현한 여러 라이브러리**



In [15]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
# 사이킷런 라이브러리가 아니지만, 사이킷런 라이브러리의 cross_validate() 사용 가능!
scores = cross_validate(xgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8824322471423747 0.8726214185237284


#### LightGBM

In [16]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9338079582727165 0.8789710890649293
